In [ ]:
!pip install simpy
import simpy
import random
import math
import numpy as np

In [ ]:
Total_Time = 100

RANDOM_SEED = 1
random.seed(RANDOM_SEED)

hospital_treating_time_rate = 6
home_treating_time_rate = 10

num_of_individuals = 0
S = 147 + 177 + 207
if(S > 1000): num_of_individuals = S
elif(S > 10): num_of_individuals = S + 1000
else: num_of_individuals = S*300
    
total_sick = 0
    
num_of_beds = math.ceil(num_of_individuals/12)
exponential_rate = num_of_individuals/300

print(num_of_individuals, exponential_rate, num_of_beds)

In [115]:
service_times = [] #Duration of the conversation between the customer and the operator (Service time)
daily_patient = []
daily_fullness = []

In [116]:
class Patient(object):
    def __init__(self, name, env, opr, isInitial):
        self.env = env
        self.name = name
        self.arrival_t = self.env.now
        self.r = random.uniform(1,2)
        self.hospital2home_treating_time_rate = 6*self.r
        self.isInitial = isInitial
        self.action = env.process(self.try_to_heal())
        self.duration = 0
    
    def try_to_heal(self):
        if(not self.isInitial and self.is_going_hospital() == False):
            print('%s\thome    \t%g\t%g/%g\t\t-' % (self.name, math.ceil(self.env.now),bed.count,num_of_beds))
            yield self.env.process(self.healing())
            print('%s\thealed  \t%g\t%g/%g\t\t%g' % (self.name, math.ceil(self.env.now),bed.count,num_of_beds,self.duration))
        else:
            with bed.request() as req:
                yield req
                if not self.isInitial:
                  print('%s\thospital\t%g\t%g/%g\t\t-' % (self.name, math.ceil(self.env.now),bed.count,num_of_beds))
                yield self.env.process(self.healing())
                print('%s\thealed  \t%g\t%g/%g\t\t%g' % (self.name, math.ceil(self.env.now),bed.count,num_of_beds,self.duration))
            
    def healing(self):
        if((self.is_going_hospital and bed.count != num_of_beds) or self.isInitial):
            self.duration = random.expovariate(1.0/hospital_treating_time_rate)
        elif(self.is_going_hospital):
            print("%s\treject  %g\t%g/%g\t\t-" %(self.name, math.ceil(self.env.now),bed.count,num_of_beds))
            self.duration = random.expovariate(1.0/self.hospital2home_treating_time_rate)
        else:
            self.duration = random.expovariate(1.0/home_treating_time_rate)
        yield self.env.timeout(self.duration)
        service_times.append(math.ceil(self.duration))
        
    def is_going_hospital(self):
        random_number = random.random()
        if(random_number > 0.2): return False
        else: return True
    

In [117]:
def patient_generator(env, bed, total_sick):
    """Generate new patients with using exponential rate!!!."""
    for i in range(Total_Time):
        sick_num_in_day = round(random.expovariate(1.0/exponential_rate))
        daily_patient.append(sick_num_in_day)
        daily_fullness.append(bed.count)
        for j in range(sick_num_in_day):
            patient = Patient('Patient %s' %(total_sick+1), env, bed, False)
            total_sick = total_sick + 1
        
        yield env.timeout(1)

In [118]:
def patient_generator_half_full(env, bed, total_sick):
    for i in range (num_of_beds // 2):
        yield env.timeout(0)
        patient = Patient('Initial %s' %(i+1), env, bed, True)
    for i in range(Total_Time):
        sick_num_in_day = round(random.expovariate(1.0/exponential_rate))
        daily_patient.append(sick_num_in_day)
        daily_fullness.append(bed.count)

        for j in range(sick_num_in_day):
            patient = Patient('Patient %s' %(total_sick+1), env, bed, False)
            total_sick = total_sick + 1
        
        yield env.timeout(1)

In [119]:
def patient_generator_full(env, bed, total_sick):
    for i in range (num_of_beds):
        yield env.timeout(0)
        patient = Patient('Initial %s' %(i+1), env, bed, True)
    for i in range(Total_Time):
        sick_num_in_day = round(random.expovariate(1.0/exponential_rate)) 
        daily_patient.append(sick_num_in_day)
        daily_fullness.append(bed.count)

        for j in range(sick_num_in_day):
            patient = Patient('Patient %s' %(total_sick+1), env, bed, False)
            total_sick = total_sick + 1
        
        yield env.timeout(1)

In [ ]:
env = simpy.Environment()
bed = simpy.Resource(env, capacity = num_of_beds)
env.process(patient_generator_half_full(env, bed, total_sick))
env.run(until = Total_Time)

In [ ]:
print(service_times)
print(str(np.mean(service_times)))

In [ ]:
print(daily_patient)
print(str(np.mean(daily_patient)))

In [ ]:
print(daily_fullness)
print(str(np.mean(daily_fullness)))
print(daily_fullness.count(0))